In [1]:
import tensorflow
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)

x_train = x_train.reshape((x_train.shape[0], 28, 28))
x_test = x_test.reshape((x_test.shape[0], 28, 28))
print(x_train.shape)

x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = np.stack((x_train,) * 3, axis=-1)
x_test = np.stack((x_test,) * 3, axis=-1)
print(x_train.shape)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

(60000, 28, 28)
(60000, 28, 28)
(60000, 28, 28, 3)


In [3]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    shear_range=0.1,
    height_shift_range=0.1,
    width_shift_range=0.1
)

datagen.fit(x_train)
augmented_train = datagen.flow(x_train, y_train, batch_size=32)

datagen.fit(x_test)
augmented_test = datagen.flow(x_test, y_test, batch_size=32)

In [4]:
model1 = Sequential()
model1.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model1.add(Dense(512, activation='relu'))
model1.add(Dense(10, activation='softmax'))

model1.layers[0].trainable=False
model1.summary()

94765736/94765736 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24,641,930
Trainable params: 1,054,218
Non-trainable params: 23,587,712
_________________________________________________________________


In [5]:
model2 = Sequential()
model2.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model2.add(Dense(512, activation='relu'))
model2.add(Dense(10, activation='softmax'))

model2.layers[0].trainable=False
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24,641,930
Trainable params: 1,054,218
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model1.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 276s 576ms/step - loss: 0.6668 - accuracy: 0.8004 - val_loss: 0.3695 - val_accuracy: 0.8878
Epoch 2/10
469/469 [==============================] - 261s 556ms/step - loss: 0.3616 - accuracy: 0.8862 - val_loss: 0.2981 - val_accuracy: 0.9120
Epoch 3/10
469/469 [==============================] - 254s 542ms/step - loss: 0.2936 - accuracy: 0.9074 - val_loss: 0.2663 - val_accuracy: 0.9176
Epoch 4/10
469/469 [==============================] - 254s 543ms/step - loss: 0.2558 - accuracy: 0.9184 - val_loss: 0.2304 - val_accuracy: 0.9282
Epoch 5/10
469/469 [==============================] - 247s 528ms/step - loss: 0.2383 - accuracy: 0.9237 - val_loss: 0.2718 - val_accuracy: 0.9100
Epoch 6/10
469/469 [==============================] - 255s 544ms/step - loss: 0.2218 - accuracy: 0.9282 - val_loss: 0.2040 - val_accuracy: 0.9349
Epoch 7/10
294/469 [=================>............] - ETA: 1:20 - loss: 0.2152 - accuracy: 0.9322

In [ ]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.fit(augmented_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

In [ ]:
predict1 = model1.predict(x_test)
plt.figure(figsize=(12, 12))
for i in range(25):
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(x_test[i], cmap=plt.cm.binary)
  plt.xlabel("real=" + str(y_test[i]) + ' predicted=' + str(np.argmax(predict1[i])))
plt.show()

In [ ]:
predict2 = model2.predict(x_test)
plt.figure(figsize=(12, 12))
for i in range(25):
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(x_test[i], cmap=plt.cm.binary)
  plt.xlabel("real=" + str(y_test[i]) + ' predicted=' + str(np.argmax(predict2[i])))
plt.show()

In [ ]:
out11 = model1.evaluate(x_test, y_test)

print("Model Accuracy: %.4f" % out11[1])
print("Model loss: %.4f" % out11[0])

out21 = model2.evaluate(x_test, y_test)

print("Model Accuracy: %.4f" % out21[1])
print("Model loss: %.4f" % out21[0])

In [ ]:
out12 = model1.evaluate(augmented_test)
# print(out12)

print("Model Accuracy: %.4f" % out12[1])
print("Model Loss: %.4f" % out12[0])

out21 = model2.evaluate(augmented_test)

print("Model Accuracy: %.4f" % out21[1])
print("Model Loss: %.4f" % out21[0])